In [1]:
from models import *
from models.pointnet2_cls_msg import *
import torch

In [2]:
import sys

# This should be the directory that contains your module, directly specified
package_directory = '/home/hani/GitHub/PCR_practice/Old_notebooks'

# Add the directory to sys.path
if package_directory not in sys.path:
    sys.path.append(package_directory)

In [3]:

# Now you can import your module as if it were in the same director
from simdata_utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
checkpoint_path_pnpp_original  = 'log/classification/pointnet2_msg_normals/checkpoints/best_model.pth'
#checkpoint_path_pnpp_mdnetc = '/home/hani/GitHub/PCR_practice/Old_notebooks/models/mdnet_40_C_pnpp_da/model_best_test.pth'

In [5]:
checkpoint_pnpp_original = torch.load(checkpoint_path_pnpp_original, map_location=torch.device('cpu'))
#checkpoint_pnpp_mdnetc = torch.load(checkpoint_path_pnpp_mdnetc, map_location=torch.device('cpu'))

In [6]:
checkpoint_pnpp_original.keys()

dict_keys(['epoch', 'instance_acc', 'class_acc', 'model_state_dict', 'optimizer_state_dict'])

In [7]:
### load the state dict of the model

model_state_dict_pnpp_original = checkpoint_pnpp_original['model_state_dict']
#model_state_dict_pnpp_mdnetc = checkpoint_pnpp_mdnetc['model_state']


In [8]:
### Load architecture of the model original
model_pnpp_original  = get_model(40,normal_channel=True)
model_pnpp_original.load_state_dict(model_state_dict_pnpp_original)

<All keys matched successfully>

In [9]:
### set the model to evaluation mode
model_pnpp_original.eval()

get_model(
  (sa1): PointNetSetAbstractionMsg(
    (conv_blocks): ModuleList(
      (0): ModuleList(
        (0): Conv2d(6, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (1): ModuleList(
        (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
      (2): ModuleList(
        (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (bn_blocks): ModuleList(
      (0): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchN

In [10]:
### loading data
# Example usage
#hdf5_file = 'partial_mdnet_40.h5'
hdf5_file = '/home/hani/GitHub/PCR_practice/Old_notebooks/mdnet_sorted_no_corr_clean.h5'
#hdf5_file = 'combined.h5'
# Reading the data
data = load_entire_dataset(hdf5_file)

print('Instances', len(list(data.keys())))

Instances 1300


In [11]:
### extract source and target point clouds ###
src_dst_data =  extract_src_dst(data)
part_dst_data = extract_partViews_dst(data)

In [12]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc
def read_data_to_numpy(filepath):
    """
    Reads a text file with point cloud data into a numpy array.

    Parameters:
    - filepath: str, the path to the text file containing the data.

    Returns:
    - data: numpy.ndarray, an array of shape (n, 6) where n is the number of points.
    """
    # Initialize an empty list to store the data points
    data_points = []

    # Open the file and read line by line
    with open(filepath, 'r') as file:
        for line in file:
            # Split each line into components, convert to float, and append to the list
            point = [float(value) for value in line.strip().split(',')]
            data_points.append(point)

    # Convert the list of data points into a numpy array
    data = np.array(data_points)

    return data

In [13]:
sample_data_path = '/home/hani/GitHub/PCR_practice/modelnet40_normal_resampled/airplane/airplane_0015.txt'
smaple_data_with_normals = read_data_to_numpy(sample_data_path)
smaple_data_with_normals_ds = downsample_data_ndarray(smaple_data_with_normals,100,25)
sample_cloud_xyz = smaple_data_with_normals_ds[:,:3]
sample_cloud_normal = smaple_data_with_normals_ds[:,3:]

In [14]:
part_source = part_dst_data['airplane_O10_V1']
part_dst = part_dst_data['airplane_O15_Vdst'] 
#part_dst = pc_normalize(part_dst)

In [15]:
#### data is loaded and ready to be used

In [32]:
additional_features = estimate_robust_normals_using_open3d(B_xyz_part_1)  # Replace with actual features if available
#additional_features = np.random.rand(part_dst.shape[0], 3)  # Replace with actual features if available


In [33]:
augmented_point_cloud_np = np.concatenate([B_xyz_part_1,additional_features], axis=1)

In [34]:
augmented_point_cloud_np.shape

(760, 6)

In [35]:
point_cloud_tensor = torch.tensor(augmented_point_cloud_np, dtype=torch.float).transpose(0, 1).unsqueeze(0)

In [36]:
point_cloud_tensor.shape

torch.Size([1, 6, 760])

In [37]:
# Make a prediction
with torch.no_grad():
    prediction, l3_pts,l2_pts,l1_pts = model_pnpp_original.forward(point_cloud_tensor)

In [38]:
prediction

tensor([[ -2.9035, -10.1638, -10.1773,  -8.5066, -10.8212, -11.2051,  -8.8252,
          -3.5607, -16.8487, -10.1059, -11.8154,  -4.5686, -14.8994, -12.4416,
         -14.5933,  -6.0149, -12.5874,  -6.0080,  -1.9921,  -9.2863, -11.1617,
         -13.3949, -11.4661, -19.5833,  -7.8436,  -8.2134,  -0.2854,  -6.6824,
         -12.5273, -10.3016,  -9.6386,  -4.7567, -15.6452, -10.6411,  -6.7339,
         -12.0475, -15.5110,  -7.6033, -14.5655,  -7.8309]])

In [39]:
_, predicted_class = torch.max(prediction, 1)  # Returns the value and index of the maximum value in each row
# or
predicted_class = prediction.argmax(1)  # Directly returns the index of the maximum value in each row

print(predicted_class)


tensor([26])


In [41]:
l3_pts.shape

torch.Size([1, 1024, 1])

In [29]:
### create partial data

def fixed_viewpoints():
    """
    Define 8 fixed viewpoints on the unit sphere using latitude and longitude.
    """
    # This is an example. You might want to refine the distribution of these points
    latitudes = [0, 0, 90, -90, 45, 45, -45, -45]
    longitudes = [0, 180, 0, 0, 45, -135, 45, -135]

    viewpoints = []
    for lat, lon in zip(latitudes, longitudes):
        lat_rad = np.radians(lat)
        lon_rad = np.radians(lon)

        x = np.cos(lat_rad) * np.cos(lon_rad)
        y = np.cos(lat_rad) * np.sin(lon_rad)
        z = np.sin(lat_rad)
        viewpoints.append([x, y, z])
    return np.array(viewpoints)

def points_in_view_frame(A_xyz, viewpoint, frame_size=1):
    """
    Select points from the point cloud that are visible within the view frame of the given viewpoint.

    Parameters:
    A_xyz (numpy.ndarray): The point cloud as an n x 3 numpy array.
    viewpoint (numpy.ndarray): A 3D point representing the viewpoint.
    frame_size (float): The size of the view frame. Smaller values mean a narrower field of view.

    Returns:
    numpy.ndarray: Points visible within the view frame.
    """
    visible_points = []
    for point in A_xyz:
        # Project point onto the view direction plane
        direction = point - viewpoint
        distance = np.linalg.norm(direction)
        if distance <= frame_size:
            visible_points.append(point)
    return np.array(visible_points)

In [30]:
### Multiple partial views ###
viewpoints = fixed_viewpoints()

# Creating 8 partial point clouds
B_xyz_part_1 = points_in_view_frame(sample_cloud_xyz, viewpoints[0]) 
B_xyz_part_2 = points_in_view_frame(sample_cloud_xyz, viewpoints[1]) 
B_xyz_part_3 = points_in_view_frame(sample_cloud_xyz, viewpoints[2]) 
B_xyz_part_4 = points_in_view_frame(sample_cloud_xyz, viewpoints[3]) 
B_xyz_part_5 = points_in_view_frame(sample_cloud_xyz, viewpoints[4]) 
B_xyz_part_6 = points_in_view_frame(sample_cloud_xyz, viewpoints[5]) 
B_xyz_part_7 = points_in_view_frame(sample_cloud_xyz, viewpoints[6]) 
B_xyz_part_8 = points_in_view_frame(sample_cloud_xyz, viewpoints[7]) 

In [31]:



viz1pcl_xyz(B_xyz_part_1)